# Computing the binary hypervolume indicator
Given two Pareto-optimal frontiers f1 and f2, this program computes the hypervolume I_H2(f1,f2) bounded by one frontier f1, not bounded by the other f2. The relative values of I_H2(f1,f2) and I_H2(f2,f1) provide information about the dominance relationship that exists between the two frontiers. The larger I_H2(f1,f2), the more volume of f1 that is not captured by f2, which signifies less conflit among the objectives in f1 relative to f2.

In [18]:
import pandas as pd
import numpy as np
import time

### Preparation for volume computation algorithm

In [19]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_None/OptimalSolutions_NONE_nondominated.txt")
sols2 = pd.read_csv("../solutionSets/3d/ClimateChange_E85/OptimalSolutions_E85_temp_nondominated.txt")

In [20]:
# Hard-coded manipulations to tidy-up sols

# set index, remove unwanted columns
sols = sols.set_index("1-" + sols["SolutionIndex"].astype(str))
sols = sols.drop(["SolutionIndex"],axis=1)
# set index, remove unwanted columns
sols2 = sols2.set_index("2-" + sols2["SolutionIndex"].astype(str))
sols2 = sols2.drop(["SolutionIndex"],axis=1)

In [21]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [22]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses[" MinOwlHabitat"] = 1
#objSenses["PER"] = 0
#objSenses

In [23]:
def maxAll(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Convert each objective to a maximization

    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]))
    
    return f

In [24]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

In [25]:
# Compute the binary epsilon indicator


# Set this boolean toggle to
    # true if you want to normalize the objective space,
    # false otherwise
normalize = False

if normalize:
    sols = maxAllAndNorm(sols,objSenses)
    sols2 = maxAllAndNorm(sols2,objSenses)
else:
    sols = maxAll(sols,objSenses)
    sols2 = maxAll(sols2,objSenses)

In [26]:
def removeDominatedSolutions(f):
    '''
    Given a dataframe of solutions, returns a dataframe with only
    nondominated solutions.
    
    It is assumed that each column is an objective and that all objectives
    aim to be maximized.
    '''
    newf = f

    objectives = f.columns.tolist()
    
    for idx,row in f.iterrows(): # searching for a solution that dominates this one
        for idx2,row2 in f.iterrows(): # scanning over all other solutions
            if np.all((row[objectives] < row2[objectives])): # if we find a solution that dominates idx,
                newf = newf.drop(idx) # then we remove that index from the to-be-returned dataframe
                break # and stop searching for a dominating solution
            
    return newf

### Algorithm to compute volume of frontier

For a mathematical description of the below algorithm, see ./algorithmWriteUp/frontierVolumeComputation_writeup.pdf

In [27]:
# Sort the dataframe in descending order from first column
# (always descending, since objectives normalized bad->good = 0->1)

sols = sols.sort_values(by=[objs[0]],ascending=False)
sols2 = sols2.sort_values(by=[objs[0]],ascending=False)
# Create the merged frontier
combinedFrontier = removeDominatedSolutions(pd.concat([sols,sols2])).sort_values(by=[objs[0]],ascending=False)

In [28]:
# Create an empty dataframe to hold the solutions whose volumes have already been accounted for,
# adding a column to hold its subdimensional volume contribution,
# and a column to indicate whether it is a boundary solution in sub-dimensional space


completedFrontierPoints = sols[sols[objs[0]]<0]
completedFrontierPoints["SubDimContribution"] = pd.Series()
completedFrontierPoints["BoundarySolution"] = pd.Series()

In [29]:
# Get list of all non-primary dimensions

dims_secondary = [col for col in sols.columns if col not in [objs[0]]]

In [30]:
def getSideVolInSubDim(dim,frontierPoint,completedFrontierPoints):
    
    sideVolInSubDim = 0
    
    # Get the sorted list of boundary solutions with dim component larger than the current point
    sideSols_dim = completedFrontierPoints.loc[(completedFrontierPoints["BoundarySolution"]>0) & \
                                              (completedFrontierPoints[dim] > frontierPoint[dim])]\
                                            .sort_values(by=[dim],ascending=True)
    # If there are none, there are no sides to add
    if sideSols_dim.empty:
        return sideVolInSubDim
    
    otherSecondaryDims = [d for d in dims_secondary if d != dim]
    prevDimComponent = frontierPoint[dim]
    currDimComponent = 0
    for idx,row in sideSols_dim.iterrows():
        currDimComponent = row[dim]
        dimDelta = currDimComponent - prevDimComponent
        sideVolInSubDim += dimDelta * row[otherSecondaryDims].product()
        prevDimComponent = currDimComponent
    
    return sideVolInSubDim
        

In [31]:
def getSubDimVolumeFromFrontierPoint(frontierPoint,completedFrontierPoints):
    # Get a solution's sub-D volume back to the origin
    subDimContrib = frontierPoint[dims_secondary].product()
    # Subtract everything pre-existing away from it
    subDimContrib -= completedFrontierPoints["SubDimContribution"].sum()
    # Add back in the sides
    for dim in dims_secondary:
        subDimContrib += getSideVolInSubDim(dim,frontierPoint,completedFrontierPoints)
    
    return subDimContrib

In [32]:
def getFrontierVolume(initFrontierVolume, remainingFrontierPoints, completedFrontierPoints):
    if len(remainingFrontierPoints) == 0:
        return initFrontierVolume
    else:
        # next solution to add to frontier vol
        currSol = remainingFrontierPoints.iloc[0]
        # will always be nondominated in sub-D space
        currSol["BoundarySolution"] = True
        # change boundary status of points that the current solution dominates:
        completedFrontierPoints.loc[np.all([completedFrontierPoints[obj] < currSol[obj] for obj in dims_secondary]\
                                           ,axis=0),\
                                    "BoundarySolution"] = False
        # Get the sub-D volume for the current solution
        currSol["SubDimContribution"] = getSubDimVolumeFromFrontierPoint(currSol,completedFrontierPoints)
        # Update the frontier volume
        initFrontierVolume += currSol["SubDimContribution"] * currSol[objs[0]]
        # update the points added to the frontier
        completedFrontierPoints = completedFrontierPoints.append(currSol,ignore_index=True)
        # update the remaining points
        remainingFrontierPoints = remainingFrontierPoints.drop(currSol.name)
        # recursive call
        return getFrontierVolume(initFrontierVolume,remainingFrontierPoints,completedFrontierPoints)

In [33]:
start = time.clock()
frontierVolume1 = getFrontierVolume(0,sols,completedFrontierPoints)
end = time.clock()
print ("Elapsed time: " + (str(end-start)))
print ("Frontier Volume 1: " + str(frontierVolume1))
start = time.clock()
frontierVolume2 = getFrontierVolume(0,sols2,completedFrontierPoints)
end = time.clock()
print ("Elapsed time: " + (str(end-start)))
print ("Frontier Volume 2: " + str(frontierVolume2))
start = time.clock()
frontierVolumeBoth = getFrontierVolume(0,combinedFrontier,completedFrontierPoints)
end = time.clock()
print ("Elapsed time: " + (str(end-start)))
print ("Frontier Volume Merged: " + str(frontierVolumeBoth))

Elapsed time: 0.6567003458005729
Frontier Volume 1: 101896.354247
Elapsed time: 1.6495067872877485
Frontier Volume 2: 3470983.74451
Elapsed time: 2.6043782303871694
Frontier Volume Merged: 3497627.87742


In [18]:
frontierVolumeBoth-frontierVolume1

17049920.132945087